In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import mobilenet_v2
from torch.utils.data import DataLoader, random_split
from collections import Counter

# Configuration du dispositif
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Préparer les données
data_dir = "CUB_200_2011/CUB_200_2011/images"

# Transformations avec augmentation des données
transform = transforms.Compose([
    transforms.RandomResizedCrop((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Charger le dataset initial
dataset = datasets.ImageFolder(data_dir, transform=transform)

# Sélection des 10 classes avec le plus d'images
class_counts = Counter(dataset.targets)
top_classes = [cls for cls, _ in class_counts.most_common(10)]
filtered_samples = [(path, label) for path, label in dataset.samples if label in top_classes]
class_mapping = {label: idx for idx, label in enumerate(top_classes)}
filtered_samples = [(path, class_mapping[label]) for path, label in filtered_samples]

# Création d'un dataset filtré
class FilteredDataset(torch.utils.data.Dataset):
    def __init__(self, samples, transform):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = datasets.folder.default_loader(path)
        if self.transform:
            image = self.transform(image)
        return image, label

filtered_dataset = FilteredDataset(filtered_samples, transform)

# Division des données en ensembles d'entraînement, validation et test
train_size = int(0.7 * len(filtered_dataset))
val_size = int(0.15 * len(filtered_dataset))
test_size = len(filtered_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(filtered_dataset, [train_size, val_size, test_size])

dataloaders = {
    'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0),
    'val': DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0),
    'test': DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)
}

# Charger MobileNetV2 avec fine-tuning
model = mobilenet_v2(weights='IMAGENET1K_V1')
model.classifier[1] = nn.Linear(model.last_channel, len(top_classes))
model = model.to(device)

# Optimisation
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Fonction d'entraînement
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print('-' * 10)
        model.train()

        running_loss = 0.0
        running_corrects = 0

        # Phase d'entraînement
        for inputs, labels in dataloaders['train']:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += (outputs.argmax(1) == labels).sum().item()

        epoch_loss = running_loss / len(dataloaders['train'].dataset)
        epoch_acc = running_corrects / len(dataloaders['train'].dataset)
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # Phase de validation après chaque époque
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in dataloaders['val']:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += (outputs.argmax(1) == labels).sum().item()

        val_loss = val_loss / len(dataloaders['val'].dataset)
        val_acc = val_corrects / len(dataloaders['val'].dataset)
        print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        scheduler.step()
    return model

# Fonction d'évaluation
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total

# Entraîner le modèle
model = train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=10)

# Évaluer sur le test
test_acc = evaluate_model(model, dataloaders['test'])
print(f"Test Accuracy: {test_acc:.4f}")


Epoch 1/10
----------
Train Loss: 1.6972 Acc: 0.3952
Val Loss: 1.0886 Acc: 0.6222
Epoch 2/10
----------
Train Loss: 0.9407 Acc: 0.6929
Val Loss: 0.9167 Acc: 0.7000
Epoch 3/10
----------
Train Loss: 0.8927 Acc: 0.6810
Val Loss: 1.0457 Acc: 0.6556
Epoch 4/10
----------
Train Loss: 0.9110 Acc: 0.7048
Val Loss: 0.9804 Acc: 0.6778
Epoch 5/10
----------
Train Loss: 0.7580 Acc: 0.7214
Val Loss: 0.9907 Acc: 0.6222
Epoch 6/10
----------
Train Loss: 0.7849 Acc: 0.7452
Val Loss: 0.9865 Acc: 0.6556
Epoch 7/10
----------
Train Loss: 0.8222 Acc: 0.7214
Val Loss: 0.9504 Acc: 0.7000
Epoch 8/10
----------
Train Loss: 0.5756 Acc: 0.7952
Val Loss: 0.8773 Acc: 0.7000
Epoch 9/10
----------
Train Loss: 0.5936 Acc: 0.7976
Val Loss: 1.0438 Acc: 0.6778
Epoch 10/10
----------
Train Loss: 0.5202 Acc: 0.8405
Val Loss: 0.7727 Acc: 0.7000
Test Accuracy: 0.7333


In [3]:
import torch
from torchvision import models
import torch.nn as nn

# Initialize MobileNetV2
num_classes = 10  # Modify to match your number of classes
model = models.mobilenet_v2(pretrained=True)

# Modify the classifier layer for your specific problem (10 output classes)
model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features, out_features=num_classes)

# Save the model state dictionary
torch.save(model.state_dict(), "bird_classifier.pth")
print("Model saved as 'bird_classifier.pth'")

# Loading the model (ensure it has the same architecture)
model = models.mobilenet_v2(pretrained=False)
model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features, out_features=num_classes)

# Load the saved weights
model.load_state_dict(torch.load("bird_classifier.pth"))
model.eval()  # Set the model to evaluation mode


Model saved as 'bird_classifier.pth'


C:\Users\paulj\AppData\Local\Temp\ipykernel_22556\1484499617.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("bird_classifier.pth"))


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=